In [3]:
#Import Libraries
import pandas as pd
import numpy as np
import os

In [5]:
#Import Dataframes\n",
path=r'C:\\Users\\north\\08-2024 Instacart Basket Analysis'
ords_prods_merge= pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_prods_merge_new clmns.pkl'))

In [7]:
ords_prods_merge.shape

(32404859, 18)

In [11]:
ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,order_number,order_weekday,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,merge_flag,busiest_days,busiest_period_of_day
0,0,2539329,1,1,2,8,NaN,First Order,196,1,0,Soda,77,7,9.0,both,Regular days,Average orders
1,0,2539329,1,1,2,8,NaN,First Order,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Regular days,Average orders
2,0,2539329,1,1,2,8,NaN,First Order,12427,3,0,Original Beef Jerky,23,19,4.4,both,Regular days,Average orders
3,0,2539329,1,1,2,8,NaN,First Order,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both,Regular days,Average orders
4,0,2539329,1,1,2,8,NaN,First Order,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Regular days,Average orders


**#2. Aggregated mean of the “order_number” column grouped by "department-id"**

In [6]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


#**3. How do results for the entire dataframe differ from those of the subset?**


**Analysis of the Results**
  - The results for the entire DataFrame are based on all available data, providing a more accurate reflection of the mean `order_number` for each `department_id`.
  - The subset, being a smaller sample (1000000), shows different mean values due to its limited scope.
  - The mean values in the entire DataFrame provide a more stable and generalizable estimate for `order_number` across departments, whereas the subset results differ due to sample size, representativeness, and variability.d variability.


**#4. Create a loyalty flag using loc**

In [11]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\north\AppData\Local\Temp\ipykernel_9516\1681540670.py:1: FutureWarning: The provided callable <function max at 0x000001383C2ACE00> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [19]:
ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,order_number,order_weekday,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,...,product_name,aisle_id,department_id,prices,merge_flag,busiest_days,busiest_period_of_day,order_frequency,user_spending,max_order
0,0,2539329,1,1,2,8,NaN,First Order,196,1,...,Soda,77,7,9.0,both,Regular days,Average orders,20.5,6.367797,10
1,0,2539329,1,1,2,8,NaN,First Order,14084,2,...,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Regular days,Average orders,20.5,6.367797,10
2,0,2539329,1,1,2,8,NaN,First Order,12427,3,...,Original Beef Jerky,23,19,4.4,both,Regular days,Average orders,20.5,6.367797,10
3,0,2539329,1,1,2,8,NaN,First Order,26088,4,...,Aged White Cheddar Popcorn,23,19,4.7,both,Regular days,Average orders,20.5,6.367797,10
4,0,2539329,1,1,2,8,NaN,First Order,26405,5,...,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Regular days,Average orders,20.5,6.367797,10


In [13]:
#Create Flag - "Loyalty Label"
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [14]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [17]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [19]:
ords_prods_merge ['loyalty_flag'].value_counts(dropna=False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

**#5. Check basic statistics of the product prices for each loyalty category**

In [21]:
#Multiple Aggregation
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

***Can you help me solve this? min/max don´t seem to be correct!?***

**#6. Create a spending flag for each user based on the average price across all their orders**

In [22]:
#Group by 'user-id' and calculate the mean price for each user
ords_prods_merge.groupby('user_id').agg({'prices': 'mean'})

,prices
user_id,
1,6.367797
2,7.515897
3,8.197727
4,8.205556
5,9.189189
...,...
206205,8.909375
206206,7.646667
206207,7.313453


In [25]:
# Create new "user_spending” column and assigns a loyalty flag to each customer
ords_prods_merge['user_spending'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\north\AppData\Local\Temp\ipykernel_9516\1635790054.py:2: FutureWarning: The provided callable <function mean at 0x000001383C2AD800> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['user_spending'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [35]:
ords_prods_merge.head(5)

,Unnamed: 0,order_id,user_id,order_number,order_weekday,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,...,aisle_id,department_id,prices,merge_flag,busiest_days,busiest_period_of_day,order_frequency,user_spending,max_order,loyalty_flag
0,0,2539329,1,1,2,8,NaN,First Order,196,1,...,77,7,9.0,both,Regular days,Average orders,20.5,6.367797,10,New customer
1,0,2539329,1,1,2,8,NaN,First Order,14084,2,...,91,16,12.5,both,Regular days,Average orders,20.5,6.367797,10,New customer
2,0,2539329,1,1,2,8,NaN,First Order,12427,3,...,23,19,4.4,both,Regular days,Average orders,20.5,6.367797,10,New customer
3,0,2539329,1,1,2,8,NaN,First Order,26088,4,...,23,19,4.7,both,Regular days,Average orders,20.5,6.367797,10,New customer
4,0,2539329,1,1,2,8,NaN,First Order,26405,5,...,54,17,1.0,both,Regular days,Average orders,20.5,6.367797,10,New customer


In [27]:
ords_prods_merge.loc[ords_prods_merge['user_spending'] < 10, 'spending_flag'] = 'Low Spender'

In [28]:
ords_prods_merge.loc[ords_prods_merge['user_spending'] >= 10, 'spending_flag'] = 'High Spender'

In [31]:
ords_prods_merge ['spending_flag'].value_counts(dropna=False)

spending_flag
Low Spender     31770614
High Spender      634245
Name: count, dtype: int64

In [43]:
#Check Output
ords_prods_merge[['user_id', 'user_spending', 'spending_flag']].head(100)

,user_id,user_spending,spending_flag
0,1,6.367797,Low Spender
1,1,6.367797,Low Spender
2,1,6.367797,Low Spender
3,1,6.367797,Low Spender
4,1,6.367797,Low Spender
...,...,...,...
95,2,7.515897,Low Spender
96,2,7.515897,Low Spender
97,2,7.515897,Low Spender
98,2,7.515897,Low Spender


In [33]:
high_spenders = ords_prods_merge[ords_prods_merge['spending_flag'] == 'High Spender']

In [34]:
high_spenders.head(5)

,Unnamed: 0,order_id,user_id,order_number,order_weekday,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,...,aisle_id,department_id,prices,merge_flag,busiest_days,busiest_period_of_day,max_order,loyalty_flag,user_spending,spending_flag
1486,160,1737705,17,1,2,13,NaN,First Order,7350,1,...,115,7,7.0,both,Regular days,Average orders,40,Regular customer,108.648299,High Spender
1487,160,1737705,17,1,2,13,NaN,First Order,47141,2,...,77,7,14.6,both,Regular days,Average orders,40,Regular customer,108.648299,High Spender
1488,160,1737705,17,1,2,13,NaN,First Order,17762,3,...,62,5,7.0,both,Regular days,Average orders,40,Regular customer,108.648299,High Spender
1489,161,1681401,17,2,5,10,3.0,Repeat Customer,7350,1,...,115,7,7.0,both,Regular days,Most orders,40,Regular customer,108.648299,High Spender
1490,161,1681401,17,2,5,10,3.0,Repeat Customer,18534,2,...,86,16,8.0,both,Regular days,Most orders,40,Regular customer,108.648299,High Spender


**#7. Instacart team wants you to determine frequent versus non-frequent customers**

In [37]:
#Group by 'user-id' and calculate the median of 'days_since_prior_order
ords_prods_merge.groupby('user_id').agg({'days_since_prior_order': 'median'})

,days_since_prior_order
user_id,
1,20.5
2,13.0
3,10.0
4,20.0
5,11.0
...,...
206205,30.0
206206,3.0
206207,16.0


In [39]:
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\north\AppData\Local\Temp\ipykernel_9516\2169400988.py:1: FutureWarning: The provided callable <function median at 0x000001383C3D3CE0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [53]:
ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,order_number,order_weekday,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,...,department_id,prices,merge_flag,busiest_days,busiest_period_of_day,order_frequency,user_spending,max_order,loyalty_flag,spending_flag
0,0,2539329,1,1,2,8,NaN,First Order,196,1,...,7,9.0,both,Regular days,Average orders,20.5,6.367797,10,New customer,Low Spender
1,0,2539329,1,1,2,8,NaN,First Order,14084,2,...,16,12.5,both,Regular days,Average orders,20.5,6.367797,10,New customer,Low Spender
2,0,2539329,1,1,2,8,NaN,First Order,12427,3,...,19,4.4,both,Regular days,Average orders,20.5,6.367797,10,New customer,Low Spender
3,0,2539329,1,1,2,8,NaN,First Order,26088,4,...,19,4.7,both,Regular days,Average orders,20.5,6.367797,10,New customer,Low Spender
4,0,2539329,1,1,2,8,NaN,First Order,26405,5,...,17,1.0,both,Regular days,Average orders,20.5,6.367797,10,New customer,Low Spender


In [55]:
#Assign the order frequency flag based on the median 'days_since_prior_order

In [41]:
ords_prods_merge.loc[ords_prods_merge['order_frequency'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [43]:
ords_prods_merge.loc[(ords_prods_merge['order_frequency'] > 10) & (ords_prods_merge['order_frequency'] <= 20), 'frequency_flag'] = 'Regular customer'

In [45]:
ords_prods_merge.loc[ords_prods_merge['order_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [47]:
#Check the result
ords_prods_merge['frequency_flag'].value_counts(dropna=False)

frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

In [49]:
ords_prods_merge[['user_id', 'days_since_prior_order', 'order_frequency', 'frequency_flag']].head(60)

,user_id,days_since_prior_order,order_frequency,frequency_flag
0,1,NaN,20.5,Non-frequent customer
1,1,NaN,20.5,Non-frequent customer
2,1,NaN,20.5,Non-frequent customer
3,1,NaN,20.5,Non-frequent customer
4,1,NaN,20.5,Non-frequent customer
5,1,15.0,20.5,Non-frequent customer
6,1,15.0,20.5,Non-frequent customer
7,1,15.0,20.5,Non-frequent customer
8,1,15.0,20.5,Non-frequent customer
9,1,15.0,20.5,Non-frequent customer


In [ ]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_grouped.pkl'))
